# State Farm Distracted Driver Detection


[State Farm Distracted Driver Detection](https://www.kaggle.com/c/state-farm-distracted-driver-detection#evaluation)

## Action Plan
### 1. Data Preparation and Preprocessing
### 2. Finetune and Train Model
### 3. Generate and Validate Predictions 
### 4. Submit predictions to Kaggle

# 2. Finetune and Train Model

## Setup 

In [1]:
%cd "~/kaggle/state-farm-driver-detection/code"
%pwd

/home/ubuntu/kaggle/state-farm-driver-detection/code


'/home/ubuntu/kaggle/state-farm-driver-detection/code'

In [2]:
#Create references to important directories we will use over and over
import os, sys
current_dir = os.getcwd()
CODE_HOME_DIR = current_dir
DATA_HOME_DIR = CODE_HOME_DIR + '/../input/'
print(CODE_HOME_DIR)
print(DATA_HOME_DIR)

/home/ubuntu/kaggle/state-farm-driver-detection/code
/home/ubuntu/kaggle/state-farm-driver-detection/code/../input/


In [3]:
#import modules
#from importlib import reload

import utils
from utils import *

import vgg16bn_ted
from vgg16bn_ted import Vgg16BN; 

%matplotlib inline

Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


#### Setup Paths

In [4]:
%cd $DATA_HOME_DIR

#Set path to sample/ path if desired
path = DATA_HOME_DIR + '/' # + '/sample/' 
results_path = path + '/results/'
train_path = path + '/train/'
valid_path = path + '/valid/'
test_path = path + '/test/'
model_path = path + '/models/'
if not os.path.exists(model_path): os.mkdir(model_path)

/home/ubuntu/kaggle/state-farm-driver-detection/input


#### Setup and save Data Classes, Labels, and Filenames

In [22]:
(trn_classes, val_classes, trn_labels, val_labels, trn_filenames, val_filenames, test_filenames) = get_classes(path)

Found 17940 images belonging to 10 classes.
Found 4484 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [23]:
save_array(results_path+'train_classes.bc', trn_classes)
save_array(results_path+'valid_classes.bc', val_classes)

In [5]:
trn_classes = np.array(load_array(results_path+'train_classes.bc'))
val_classes = np.array(load_array(results_path+'valid_classes.bc'))
trn_labels = onehot(trn_classes)
val_labels = onehot(val_classes)

In [25]:
save_array(results_path+'train_filenames.bc', trn_filenames)
save_array(results_path+'valid_filenames.bc', val_filenames)
save_array(results_path+'test_filenames.bc', test_filenames)

In [6]:
trn_filenames = load_array(results_path+'train_filenames.bc')
val_filenames = load_array(results_path+'valid_filenames.bc')
test_filenames = load_array(results_path+'test_filenames.bc')

In [40]:
trn_data = get_data(train_path)
val_data = get_data(valid_path)

Found 17940 images belonging to 10 classes.
Found 4484 images belonging to 10 classes.


In [41]:
save_array(results_path+'trn_data.bc', trn_data)
save_array(results_path+'val_data.bc', val_data)

In [17]:
trn_data = load_array(results_path+'trn_data.bc')
val_data = load_array(results_path+'val_data.bc')

## Method 1: First training by full Vgg model

In [5]:
vgg = Vgg16BN(size=(256, 256))
#vgg = Vgg16BN()
model = vgg.model

In [6]:
batch_size=64

In [7]:
#Finetune the model
batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, shuffle=False, batch_size=batch_size)
vgg.finetune(batches)

Found 17940 images belonging to 10 classes.
Found 4484 images belonging to 10 classes.


#### Model Tuning

In [9]:
vgg.fit(batches, val_batches, epochs=2)

Epoch 1/2
 - 346s - loss: 1.1656 - acc: 0.6431 - val_loss: 0.1348 - val_acc: 0.9674
Epoch 2/2
 - 337s - loss: 0.4400 - acc: 0.8605 - val_loss: 0.0854 - val_acc: 0.9784


In [10]:
model.optimizer.lr = 0.01

In [11]:
vgg.fit(batches, val_batches, epochs=5)

Epoch 1/5
 - 336s - loss: 0.3565 - acc: 0.8824 - val_loss: 0.0634 - val_acc: 0.9837
Epoch 2/5
 - 337s - loss: 0.3394 - acc: 0.8917 - val_loss: 0.0738 - val_acc: 0.9804
Epoch 3/5
 - 337s - loss: 0.3223 - acc: 0.8993 - val_loss: 0.0566 - val_acc: 0.9855
Epoch 4/5
 - 337s - loss: 0.3240 - acc: 0.8959 - val_loss: 0.0637 - val_acc: 0.9819
Epoch 5/5
 - 337s - loss: 0.3366 - acc: 0.8988 - val_loss: 0.0529 - val_acc: 0.9868


In [9]:
model.optimizer.lr = 0.001

In [10]:
vgg.fit(batches, val_batches, epochs=10)

Epoch 1/5
 - 337s - loss: 0.3501 - acc: 0.8957 - val_loss: 0.0541 - val_acc: 0.9853
Epoch 2/5
 - 328s - loss: 0.3587 - acc: 0.8964 - val_loss: 0.0535 - val_acc: 0.9857
Epoch 3/5
 - 328s - loss: 0.3578 - acc: 0.9004 - val_loss: 0.0491 - val_acc: 0.9866
Epoch 4/5
 - 328s - loss: 0.3615 - acc: 0.8988 - val_loss: 0.0533 - val_acc: 0.9873
Epoch 5/5
 - 328s - loss: 0.3657 - acc: 0.9002 - val_loss: 0.0581 - val_acc: 0.9851


In [11]:
model.save_weights(model_path+'vgg16bn-first1.h5')

In [8]:
model.load_weights(model_path+'vgg16bn-first1.h5')

## Method 2: Pretrain Convolution Layers to speed up and focus tuning FC Layers 

**<font color='red'>Notices</font>**: 

- <font color='red'>**here because we use this BatchNormalization dense layers based on pretrained convolution layers, we can not use Data Augmentation, because with Data Augmentation the output of pretrained convolution layers will be different every time. **
</font>

#### Setup Models

In [10]:
vgg = Vgg16BN(size=(256, 256))
#vgg = Vgg16BN()
model = vgg.model

In [11]:
batch_size=64

In [12]:
#Finetune the model
vgg.finetune(vgg.get_batches(train_path, shuffle=False, batch_size=batch_size))

Found 17940 images belonging to 10 classes.


In [13]:
conv_model, fc_model = get_split_models(model)

### 2.1 Let's pretrain the conv_model first (or load conv_model train features)


**<br><font color='red'>Notices</font>**: 
  - ** <font color='red'>batches shuffle must be set to False when pre-computing features </font>**


#### Train and save Pretrained conv_model Feature Maps

In [46]:
batches = get_batches(train_path, batch_size=batch_size, shuffle=False)
val_batches = get_batches(valid_path, batch_size=batch_size*2, shuffle=False)
test_batches = get_batches(test_path, batch_size=batch_size, shuffle=False)

Found 17940 images belonging to 10 classes.
Found 4484 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [47]:
conv_trn_features = conv_model.predict_generator(batches, steps=ceil(batches.n/batches.batch_size))
conv_val_features = conv_model.predict_generator(val_batches, steps=ceil(val_batches.n/val_batches.batch_size))
con_test_features = conv_model.predict_generator(test_batches, steps=ceil(test_batches.n/test_batches.batch_size))

In [48]:
save_array(results_path+'train_convlayer_features.dat', conv_trn_features)
save_array(results_path+'valid_convlayer_features2.dat', conv_val_features)
save_array(results_path+'test_convlayer_features.dat', conv_test_features)

In [7]:
conv_trn_features = load_array(results_path+'train_convlayer_features.dat')
conv_val_features = load_array(results_path+'valid_convlayer_features2.dat')
conv_test_features = load_array(results_path+'test_convlayer_features.dat')

In [8]:
print(conv_trn_features.shape)
print(conv_val_features.shape)
print(conv_test_features.shape)

(17940, 512, 8, 8)
(4484, 512, 8, 8)
(79726, 512, 8, 8)


### 2.2 Start training FC model

#### Model Tuning

In [51]:
fc_model.fit(conv_trn_features, trn_labels, batch_size=batch_size, epochs=2, 
             validation_data= (conv_val_features, val_labels), verbose=2)

Train on 17940 samples, validate on 4484 samples
Epoch 1/2
 - 62s - loss: 0.4260 - acc: 0.9017 - val_loss: 0.1135 - val_acc: 0.9737
Epoch 2/2
 - 60s - loss: 0.0785 - acc: 0.9815 - val_loss: 0.0498 - val_acc: 0.9891


In [52]:
fc_model.lr = 0.1

In [53]:
fc_model.fit(conv_trn_features, trn_labels, batch_size=batch_size, epochs=3, 
             validation_data= (conv_val_features, val_labels), verbose=2)

Train on 17940 samples, validate on 4484 samples
Epoch 1/3
 - 60s - loss: 0.0904 - acc: 0.9799 - val_loss: 0.0968 - val_acc: 0.9822
Epoch 2/3
 - 60s - loss: 0.0822 - acc: 0.9857 - val_loss: 0.1391 - val_acc: 0.9853
Epoch 3/3
 - 60s - loss: 0.0499 - acc: 0.9896 - val_loss: 0.0584 - val_acc: 0.9900


In [34]:
fc_model.lr = 0.01

In [35]:
fc_model.fit(conv_trn_features, trn_labels, batch_size=batch_size, epochs=2, 
             validation_data= (conv_val_features, val_labels), verbose=2)

Train on 17940 samples, validate on 4484 samples
Epoch 1/10
 - 60s - loss: 0.0487 - acc: 0.9910 - val_loss: 0.0494 - val_acc: 0.9924
Epoch 2/10
 - 60s - loss: 0.0414 - acc: 0.9929 - val_loss: 0.0348 - val_acc: 0.9955
Epoch 3/10
 - 60s - loss: 0.0483 - acc: 0.9918 - val_loss: 0.0598 - val_acc: 0.9915
Epoch 4/10
 - 60s - loss: 0.0486 - acc: 0.9924 - val_loss: 0.0969 - val_acc: 0.9875
Epoch 5/10
 - 60s - loss: 0.0520 - acc: 0.9914 - val_loss: 0.0579 - val_acc: 0.9909
Epoch 6/10
 - 60s - loss: 0.0630 - acc: 0.9907 - val_loss: 0.0669 - val_acc: 0.9931
Epoch 7/10
 - 60s - loss: 0.0730 - acc: 0.9898 - val_loss: 0.0741 - val_acc: 0.9920
Epoch 8/10
 - 60s - loss: 0.0551 - acc: 0.9923 - val_loss: 0.0790 - val_acc: 0.9902
Epoch 9/10
 - 60s - loss: 0.0367 - acc: 0.9952 - val_loss: 0.0662 - val_acc: 0.9933
Epoch 10/10
 - 60s - loss: 0.0248 - acc: 0.9960 - val_loss: 0.0217 - val_acc: 0.9978


In [36]:
fc_model.lr = 0.001

In [37]:
fc_model.fit(conv_trn_features, trn_labels, batch_size=batch_size, epochs=10, 
             validation_data= (conv_val_features, val_labels), verbose=2)

Train on 17940 samples, validate on 4484 samples
Epoch 1/10
 - 60s - loss: 0.0278 - acc: 0.9958 - val_loss: 0.0586 - val_acc: 0.9938
Epoch 2/10
 - 60s - loss: 0.0408 - acc: 0.9947 - val_loss: 0.0366 - val_acc: 0.9960
Epoch 3/10
 - 60s - loss: 0.0513 - acc: 0.9939 - val_loss: 0.0610 - val_acc: 0.9940
Epoch 4/10
 - 60s - loss: 0.0599 - acc: 0.9924 - val_loss: 0.0469 - val_acc: 0.9962
Epoch 5/10
 - 60s - loss: 0.0461 - acc: 0.9943 - val_loss: 0.0462 - val_acc: 0.9958
Epoch 6/10
 - 60s - loss: 0.0308 - acc: 0.9961 - val_loss: 0.0365 - val_acc: 0.9973
Epoch 7/10
 - 60s - loss: 0.0613 - acc: 0.9935 - val_loss: 0.0492 - val_acc: 0.9949
Epoch 8/10
 - 60s - loss: 0.0447 - acc: 0.9944 - val_loss: 0.0550 - val_acc: 0.9949
Epoch 9/10
 - 60s - loss: 0.0355 - acc: 0.9955 - val_loss: 0.0664 - val_acc: 0.9940
Epoch 10/10
 - 60s - loss: 0.0349 - acc: 0.9957 - val_loss: 0.0496 - val_acc: 0.9953


In [38]:
model_name = 'Vgg16BN-ted'
sub_model_name = 'fc-model'
fc_model.save_weights(model_path + model_name + '_' + sub_model_name + '1.h5')

In [ ]:
model_name = 'Vgg16BN-ted'
sub_model_name = 'fc-model'
fc_model.load_weights(model_path + model_name + '_' + sub_model_name + '1.h5')

### 2.3 Pre-computed data augmentation

#### Setup: use our optimized data augmentation parameters in 2.a to create a 6x blended training datasets (5x data aug + 1x raw) 
- ** Notice that here because we use this BatchNormalization dense layers based on pretrained convolution layers, we can not use Data Augmentation, because with Data Augmentation the output of pretrained convolution layers will be different every time **
- ** But you could compute 10x bigger than your dataset like 10 different versions of it. This is why I put it here with data augmentation generator and then predict it like 5 times bigger of data. This would give me 5 times of data. This actually works pretty well. It’s not as good as giving a whole new sample every time but it’s kind of a compromise **

In [9]:
batch_size=64

In [10]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
aug_batches = get_batches(train_path, gen_t, batch_size=batch_size)

Found 17940 images belonging to 10 classes.


In [ ]:
# We create a dataset of convolutional features 5x bigger than the training set.
conv_aug_features = conv_model.predict_generator(aug_batches, steps= 5 * ceil(aug_batches.n / (aug_batches.batch_size)))

In [93]:
save_array(results_path+'conv_aug_features.bc', conv_aug_features)

In [11]:
conv_aug_features = load_array(results_path+'conv_aug_features.bc')

In [12]:
print(conv_aug_features.shape)
print(conv_trn_features.shape)

(89700, 512, 8, 8)
(17940, 512, 8, 8)


In [13]:
# include the real training data as well in its non-augmented form.
conv_aug_features = np.concatenate([conv_aug_features, conv_trn_features])

# Since we've now got a dataset 6x bigger than before, we'll need to copy our labels 6 times too.
aug_trn_labels = np.concatenate([trn_labels]*6)

In [17]:
save_array(results_path+'conv_aug_features2.bc', conv_aug_features)
save_array(results_path+'aug_trn_labels.bc', aug_trn_labels)

In [18]:
conv_aug_features = load_array(results_path+'conv_aug_features2.bc')
aug_trn_labels = load_array(results_path+'aug_trn_labels.bc')

In [19]:
print(conv_aug_features.shape)
print(aug_trn_labels.shape)

(107640, 512, 8, 8)
(107640, 10)


#### Setup Models

In [20]:
vgg = Vgg16BN(size=(256, 256))
model = vgg.model

In [21]:
#Finetune the model
vgg.finetune(vgg.get_batches(train_path, shuffle=False, batch_size=batch_size))

Found 17940 images belonging to 10 classes.


In [22]:
_, fc_model = get_split_models(model)

#### Model Tuning: FC model

In [23]:
fc_model.fit(conv_aug_features, aug_trn_labels, batch_size=batch_size, epochs=2, 
             validation_data= (conv_val_features, val_labels), verbose=2)

Train on 107640 samples, validate on 4484 samples
Epoch 1/2
 - 379s - loss: 2.6552 - acc: 0.1894 - val_loss: 0.1758 - val_acc: 0.9456
Epoch 2/2
 - 373s - loss: 2.1201 - acc: 0.2355 - val_loss: 0.1750 - val_acc: 0.9596


In [24]:
fc_model.lr = 1

In [25]:
fc_model.fit(conv_aug_features, aug_trn_labels, batch_size=batch_size, epochs=30, 
             validation_data= (conv_val_features, val_labels), verbose=2)

Train on 107640 samples, validate on 4484 samples
Epoch 1/30
 - 372s - loss: 2.0345 - acc: 0.2434 - val_loss: 0.0602 - val_acc: 0.9826
Epoch 2/30
 - 372s - loss: 2.0075 - acc: 0.2458 - val_loss: 0.0717 - val_acc: 0.9904
Epoch 3/30
 - 372s - loss: 1.9833 - acc: 0.2498 - val_loss: 0.0775 - val_acc: 0.9875
Epoch 4/30
 - 372s - loss: 1.9652 - acc: 0.2576 - val_loss: 0.2880 - val_acc: 0.9732
Epoch 5/30
 - 372s - loss: 1.9501 - acc: 0.2652 - val_loss: 0.1050 - val_acc: 0.9884
Epoch 6/30
 - 372s - loss: 1.9302 - acc: 0.2788 - val_loss: 0.1149 - val_acc: 0.9893
Epoch 7/30
 - 372s - loss: 1.8987 - acc: 0.2988 - val_loss: 0.0898 - val_acc: 0.9911
Epoch 8/30
 - 372s - loss: 1.8558 - acc: 0.3201 - val_loss: 0.0601 - val_acc: 0.9931
Epoch 9/30
 - 372s - loss: 1.7958 - acc: 0.3515 - val_loss: 0.0948 - val_acc: 0.9897
Epoch 10/30
 - 372s - loss: 1.7336 - acc: 0.3813 - val_loss: 0.0839 - val_acc: 0.9911
Epoch 11/30
 - 372s - loss: 1.6566 - acc: 0.4144 - val_loss: 0.1348 - val_acc: 0.9871
Epoch 12/30
 

In [ ]:
fc_model.lr = 0.1

In [ ]:
fc_model.fit(conv_aug_features, aug_trn_labels, batch_size=batch_size, epochs=30, 
             validation_data= (conv_val_features, val_labels), verbose=2)

Train on 107640 samples, validate on 4484 samples
Epoch 1/30
 - 372s - loss: 0.4187 - acc: 0.8578 - val_loss: 0.0619 - val_acc: 0.9940
Epoch 2/30
 - 372s - loss: 0.3997 - acc: 0.8645 - val_loss: 0.0788 - val_acc: 0.9935
Epoch 3/30
 - 372s - loss: 0.3867 - acc: 0.8688 - val_loss: 0.0590 - val_acc: 0.9946
Epoch 4/30
 - 371s - loss: 0.3656 - acc: 0.8754 - val_loss: 0.0560 - val_acc: 0.9949
Epoch 5/30
 - 372s - loss: 0.3547 - acc: 0.8787 - val_loss: 0.0469 - val_acc: 0.9940
Epoch 6/30
 - 372s - loss: 0.3422 - acc: 0.8837 - val_loss: 0.0619 - val_acc: 0.9931
Epoch 7/30
 - 372s - loss: 0.3066 - acc: 0.8952 - val_loss: 0.0681 - val_acc: 0.9931
Epoch 10/30
 - 371s - loss: 0.3041 - acc: 0.8963 - val_loss: 0.0631 - val_acc: 0.9933
Epoch 11/30
 - 372s - loss: 0.2887 - acc: 0.9018 - val_loss: 0.0616 - val_acc: 0.9938
Epoch 12/30
 - 372s - loss: 0.2783 - acc: 0.9056 - val_loss: 0.0635 - val_acc: 0.9946
Epoch 13/30
 - 372s - loss: 0.2817 - acc: 0.9047 - val_loss: 0.0533 - val_acc: 0.9951
Epoch 14/30

In [ ]:
fc_model.lr = 0.01

In [ ]:
fc_model.fit(conv_aug_features, aug_trn_labels, batch_size=batch_size, epochs=30, 
             validation_data= (conv_val_features, val_labels), verbose=2)

Train on 107640 samples, validate on 4484 samples
Epoch 1/30
 - 372s - loss: 0.1806 - acc: 0.9390 - val_loss: 0.0599 - val_acc: 0.9935
Epoch 2/30
 - 372s - loss: 0.1767 - acc: 0.9402 - val_loss: 0.0586 - val_acc: 0.9938
Epoch 3/30
 - 372s - loss: 0.1761 - acc: 0.9407 - val_loss: 0.0440 - val_acc: 0.9955
Epoch 4/30
 - 372s - loss: 0.1694 - acc: 0.9428 - val_loss: 0.0651 - val_acc: 0.9940
Epoch 5/30
 - 372s - loss: 0.1672 - acc: 0.9432 - val_loss: 0.0556 - val_acc: 0.9944
Epoch 6/30
 - 372s - loss: 0.1681 - acc: 0.9429 - val_loss: 0.0531 - val_acc: 0.9946
Epoch 7/30
 - 372s - loss: 0.1649 - acc: 0.9445 - val_loss: 0.0451 - val_acc: 0.9949
Epoch 8/30
 - 372s - loss: 0.1537 - acc: 0.9481 - val_loss: 0.0485 - val_acc: 0.9953
Epoch 9/30
 - 372s - loss: 0.1568 - acc: 0.9471 - val_loss: 0.0702 - val_acc: 0.9940
Epoch 10/30
 - 372s - loss: 0.1579 - acc: 0.9465 - val_loss: 0.0979 - val_acc: 0.9924
Epoch 11/30
 - 372s - loss: 0.1526 - acc: 0.9484 - val_loss: 0.0577 - val_acc: 0.9933
Epoch 12/30
 

In [ ]:
fc_model.lr = 0.001

In [ ]:
fc_model.fit(conv_aug_features, aug_trn_labels, batch_size=batch_size, epochs=50, 
             validation_data= (conv_val_features, val_labels), verbose=2)

Train on 107640 samples, validate on 4484 samples
Epoch 1/50
 - 372s - loss: 0.1209 - acc: 0.9598 - val_loss: 0.0698 - val_acc: 0.9940
Epoch 2/50
 - 372s - loss: 0.1147 - acc: 0.9619 - val_loss: 0.0515 - val_acc: 0.9958
Epoch 3/50
 - 372s - loss: 0.1146 - acc: 0.9609 - val_loss: 0.0617 - val_acc: 0.9942
Epoch 4/50
 - 372s - loss: 0.1138 - acc: 0.9626 - val_loss: 0.0472 - val_acc: 0.9960
Epoch 5/50
 - 372s - loss: 0.1142 - acc: 0.9618 - val_loss: 0.0531 - val_acc: 0.9951
Epoch 6/50
 - 372s - loss: 0.1096 - acc: 0.9628 - val_loss: 0.0633 - val_acc: 0.9944
Epoch 7/50
 - 372s - loss: 0.1103 - acc: 0.9641 - val_loss: 0.0587 - val_acc: 0.9949
Epoch 8/50
 - 372s - loss: 0.1082 - acc: 0.9634 - val_loss: 0.0527 - val_acc: 0.9958
Epoch 9/50
 - 372s - loss: 0.1054 - acc: 0.9647 - val_loss: 0.0544 - val_acc: 0.9953
Epoch 10/50
 - 372s - loss: 0.1096 - acc: 0.9627 - val_loss: 0.0502 - val_acc: 0.9946
Epoch 11/50
 - 372s - loss: 0.1055 - acc: 0.9647 - val_loss: 0.0703 - val_acc: 0.9949
Epoch 12/50
 

In [34]:
fc_model.lr = 0.0001

In [35]:
fc_model.fit(conv_aug_features, aug_trn_labels, batch_size=batch_size, epochs=5, 
             validation_data= (conv_val_features, val_labels), verbose=2)

Train on 107640 samples, validate on 4484 samples
Epoch 1/5
 - 372s - loss: 0.0739 - acc: 0.9753 - val_loss: 0.0671 - val_acc: 0.9944
Epoch 2/5
 - 372s - loss: 0.0754 - acc: 0.9747 - val_loss: 0.0673 - val_acc: 0.9938
Epoch 3/5
 - 373s - loss: 0.0745 - acc: 0.9754 - val_loss: 0.0591 - val_acc: 0.9949
Epoch 4/5
 - 372s - loss: 0.0719 - acc: 0.9760 - val_loss: 0.0654 - val_acc: 0.9946
Epoch 5/5
 - 372s - loss: 0.0735 - acc: 0.9756 - val_loss: 0.0577 - val_acc: 0.9953


In [36]:
fc_model.lr = 0.001

In [37]:
fc_model.fit(conv_aug_features, aug_trn_labels, batch_size=batch_size, epochs=3, 
             validation_data= (conv_val_features, val_labels), verbose=2)

Train on 107640 samples, validate on 4484 samples
Epoch 1/3
 - 372s - loss: 0.0723 - acc: 0.9760 - val_loss: 0.0663 - val_acc: 0.9949
Epoch 2/3
 - 372s - loss: 0.0702 - acc: 0.9768 - val_loss: 0.0654 - val_acc: 0.9944
Epoch 3/3
 - 372s - loss: 0.0730 - acc: 0.9758 - val_loss: 0.0631 - val_acc: 0.9946


In [38]:
fc_model.lr = 0.01

In [39]:
fc_model.fit(conv_aug_features, aug_trn_labels, batch_size=batch_size, epochs=3, 
             validation_data= (conv_val_features, val_labels), verbose=2)

Train on 107640 samples, validate on 4484 samples
Epoch 1/3
 - 372s - loss: 0.0710 - acc: 0.9769 - val_loss: 0.0607 - val_acc: 0.9951
Epoch 2/3
 - 372s - loss: 0.0718 - acc: 0.9763 - val_loss: 0.0475 - val_acc: 0.9953
Epoch 3/3
 - 372s - loss: 0.0697 - acc: 0.9768 - val_loss: 0.0597 - val_acc: 0.9953


In [40]:
model_name = 'Vgg16BN-ted'
sub_model_name = 'fc-model'
fc_model.save_weights(model_path + model_name + '_' + sub_model_name + '_aug2.h5')

In [41]:
model_name = 'Vgg16BN-ted'
sub_model_name = 'fc-model'
fc_model.load_weights(model_path + model_name + '_' + sub_model_name + '_aug2.h5')

### 2.4 Regularization

** Regularization can not be done with only sample, but need to be run on full dataset. It's because Regularization is very correlated with data.** (Running on full dataset, we will need much less regularization than on only sample dataset.)

### 2.5 Pseudo labeling

#### The rule of thumb is ¼ ~ 1/3 of your batches be peudo-labeling. 
#### Setup: put pseudo labeling on unlabeled test data and add to training set by 1:1

In [ ]:
test_pseudo_labels = fc_model.predict(conv_test_feature, batch_size=batch_size)

In [ ]:
# concatenate them with our training and val labels
comb_pseudo_labels = np.concatenate([aug_trn_labels, test_pseudo_labels])
comb_pseudo_features = np.concatenate([conv_aug_features, conv_test_feature])

#### Setup Models

In [73]:
vgg = Vgg16BN(size=(256, 256))
model = vgg.model

In [74]:
batch_size=64

In [75]:
#Finetune the model
vgg.finetune(vgg.get_batches(train_path, shuffle=False, batch_size=batch_size))

Found 17940 images belonging to 10 classes.


In [83]:
_, fc_model = get_split_models(model)

#### Model Tuning: FC model

In [ ]:
fc_model.fit(comb_pseudo_features, comb_pseudo_labels, batch_size=batch_size, epochs=2, 
             validation_data= (conv_val_features, val_labels), verbose=2)